In [60]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [61]:
!pip install tensorflow_datasets
!pip install tensorflow_text

In [62]:
import collections
import logging
import os
import pathlib 
import re 
import string
import sys 
import time 

import numpy as np
import matplotlib.pyplot as plt 

import tensorflow_datasets as tfds 
import tensorflow_text as text 
import tensorflow as tf

from drive.MyDrive.transformer.transformer import Transformer
from drive.MyDrive.transformer.scheduler import CustomSchedule

In [63]:
logging.getLogger('tensorflow').setLevel(logging.ERROR)

In [64]:
examples, metadata = tfds.load('ted_hrlr_translate/ru_to_en', with_info=True,
                               as_supervised=True)
train_examples, val_examples = examples['train'], examples['validation']

In [65]:
model_name = 'drive/MyDrive/ru_en_bert_converter'
tokenizers = tf.saved_model.load(model_name)

In [66]:
def tokenize_pairs(ru, en):
    ru = tokenizers.ru.tokenize(ru)
    ru = ru.to_tensor()

    en = tokenizers.en.tokenize(en)
    en = en.to_tensor()
    return en, ru

In [67]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64
EPOCHS = 10

In [68]:
def make_batches(ds):
  return (
      ds
      .cache()
      .shuffle(BUFFER_SIZE)
      .batch(BATCH_SIZE)
      .map(tokenize_pairs, num_parallel_calls=tf.data.AUTOTUNE)
      .prefetch(tf.data.AUTOTUNE))
    
train_batches = make_batches(train_examples)
val_batches = make_batches(val_examples)

In [69]:
num_layers = 4
d_model = 128
dff = 512
num_heads = 8
dropout_rate = 0.1

In [70]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [71]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.Mean(name='train_accuracy')

In [72]:
transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=tokenizers.en.get_vocab_size().numpy(),
    target_vocab_size=tokenizers.ru.get_vocab_size().numpy(),
    pe_input=1000,
    pe_target=1000,
    rate=dropout_rate)

In [73]:
checkpoint_path = "./checkpoints/train"

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

In [74]:
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)


def accuracy_function(real, pred):
    accuracies = tf.equal(real, tf.argmax(pred, axis=2))

    mask = tf.math.logical_not(tf.math.equal(real, 0))
    accuracies = tf.math.logical_and(mask, accuracies)

    accuracies = tf.cast(accuracies, dtype=tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    return tf.reduce_sum(accuracies)/tf.reduce_sum(mask)

In [75]:
train_step_signature = [
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
]


@tf.function(input_signature=train_step_signature)
def train_step(inp, tar):
  tar_inp = tar[:, :-1]
  tar_real = tar[:, 1:]

  with tf.GradientTape() as tape:
    predictions = transformer([inp, tar_inp],
                                 training = True)
    loss = loss_function(tar_real, predictions)

  gradients = tape.gradient(loss, transformer.trainable_variables)
  optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

  train_loss(loss)
  train_accuracy(accuracy_function(tar_real, predictions))

In [76]:
for epoch in range(EPOCHS):
  start = time.time()

  train_loss.reset_states()
  train_accuracy.reset_states()

  for (batch, (inp, tar)) in enumerate(train_batches):
    train_step(inp, tar)

    if batch % 50 == 0:
      print(f'Epoch {epoch + 1} Batch {batch} Loss {train_loss.result():.4f} Accuracy {train_accuracy.result():.4f}')

  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print(f'Saving checkpoint for epoch {epoch+1} at {ckpt_save_path}')

  print(f'Epoch {epoch + 1} Loss {train_loss.result():.4f} Accuracy {train_accuracy.result():.4f}')

  print(f'Time taken for 1 epoch: {time.time() - start:.2f} secs\n')

Epoch 1 Batch 0 Loss 8.9740 Accuracy 0.0000
Epoch 1 Batch 50 Loss 8.9225 Accuracy 0.0186
Epoch 1 Batch 100 Loss 8.8307 Accuracy 0.0390
Epoch 1 Batch 150 Loss 8.7324 Accuracy 0.0454
Epoch 1 Batch 200 Loss 8.6114 Accuracy 0.0487
Epoch 1 Batch 250 Loss 8.4679 Accuracy 0.0510
Epoch 1 Batch 300 Loss 8.3119 Accuracy 0.0522
Epoch 1 Batch 350 Loss 8.1535 Accuracy 0.0560
Epoch 1 Batch 400 Loss 8.0043 Accuracy 0.0612
Epoch 1 Batch 450 Loss 7.8735 Accuracy 0.0665
Epoch 1 Batch 500 Loss 7.7609 Accuracy 0.0711
Epoch 1 Batch 550 Loss 7.6601 Accuracy 0.0752
Epoch 1 Batch 600 Loss 7.5669 Accuracy 0.0792
Epoch 1 Batch 650 Loss 7.4809 Accuracy 0.0828
Epoch 1 Batch 700 Loss 7.4005 Accuracy 0.0861
Epoch 1 Batch 750 Loss 7.3246 Accuracy 0.0893
Epoch 1 Batch 800 Loss 7.2531 Accuracy 0.0924
Epoch 1 Batch 850 Loss 7.1860 Accuracy 0.0953
Epoch 1 Batch 900 Loss 7.1230 Accuracy 0.0980
Epoch 1 Batch 950 Loss 7.0635 Accuracy 0.1006
Epoch 1 Batch 1000 Loss 7.0079 Accuracy 0.1031
Epoch 1 Batch 1050 Loss 6.9560 Accur

In [77]:
class Translator(tf.Module):
  def __init__(self, tokenizers, transformer):
    self.tokenizers = tokenizers
    self.transformer = transformer

  def __call__(self, sentence, max_length=20):
    assert isinstance(sentence, tf.Tensor)
    if len(sentence.shape) == 0:
      sentence = sentence[tf.newaxis]

    sentence = self.tokenizers.en.tokenize(sentence).to_tensor()

    encoder_input = sentence

    start_end = self.tokenizers.en.tokenize([''])[0]
    start = start_end[0][tf.newaxis]
    end = start_end[1][tf.newaxis]

    output_array = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)
    output_array = output_array.write(0, start)

    for i in tf.range(max_length):
        output = tf.transpose(output_array.stack())
        predictions = self.transformer([encoder_input, output], training=False)

        predictions = predictions[:, -1:, :]

        predicted_id = tf.argmax(predictions, axis=-1)

        output_array = output_array.write(i+1, predicted_id[0])

        if predicted_id == end:
            break

    output = tf.transpose(output_array.stack())
    text = self.tokenizers.ru.detokenize(output)[0]

    tokens = self.tokenizers.ru.lookup(output)[0]

    return text, tokens

In [78]:
translator = Translator(tokenizers, transformer)

In [79]:
def print_translation(sentence, tokens, ground_truth):
  print(f'{"Input:":15s}: {sentence}')
  print(f'{"Prediction":15s}: {tokens.numpy().decode("utf-8")}')
  print(f'{"Ground truth":15s}: {ground_truth}')

In [80]:
sentence = "Hi."
ground_truth = "Привет."

translated_text, translated_tokens= translator(
    tf.constant(sentence))
print_translation(sentence, translated_text, ground_truth)

Input:         : Hi.
Prediction     : привет .
Ground truth   : Привет.


In [81]:
class ExportTranslator(tf.Module):
  def __init__(self, translator):
    self.translator = translator

  @tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.string)])
  def __call__(self, sentence):
    (result, tokens) = self.translator(sentence, max_length=100)

    return result

In [82]:
translator = ExportTranslator(translator)

In [83]:
translator("How are you?").numpy().decode("utf-8")


'как вы ?'

In [84]:
tf.saved_model.save(translator, export_dir='eng_ru_translator')

In [85]:
reloaded = tf.saved_model.load('eng_ru_translator')

In [86]:
reloaded("What do you do?").numpy().decode('utf-8')

'что делать ?'